<a href="https://colab.research.google.com/github/FabiolaAJ/chatbot-assistente-de-vagas/blob/main/Chatbot_Assistente_de_Vagas_de_emprego.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
import os
from google.colab import userdata
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
from IPython.display import clear_output

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [162]:
##########################################
# --- Agente 1: Buscador de Vagas --- #
##########################################
def agente_buscador_vagas_de_emprego(profissao, area, senioridade, data_de_hoje):

    buscador = Agent(
        name="agente_buscador_de_vagas",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente simpático especializado na busca de vagas de emprego.
        Use a ferramenta de busca do google (google_search) para encontrar até 5 vagas relevantes e atuais para a profissão e senioridade do usuário.

          1. **Interpretação:** Analise profissão e senioridade. Se a senioridade faltar, pergunte.
          2. **Busca (google_search):** Combine profissão, área, senioridade e informe se a vaga é hibrida, presencial ou remoto. Foque em resultados que provavelmente contenham links diretos para a página da vaga.
          3. **Filtragem:** Priorize vagas recentes verificando a data de hoje (data_de_hoje). Descarte vagas antigas ou encerradas.
          4. **Resposta (máximo 5 vagas):** Para cada vaga, retorne título, empresa, localização (se disponível), breve descrição com requisitos para vaga, não faça questionamentos adicionais para o usuário.
          5. **Link:** [**Link direto para a página da vaga**. Este é um campo obrigatório. Tente ao máximo encontrar o link nos resultados da busca.]
          6. **Sem Resultados:** Se não houver vagas relevantes e atuais, informe o usuário.

          Priorize informações precisas e atuais.

        """,
        description="Agente que busca informações de vagas de empregos no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Profissão: {profissao} \n Area:{area} \n Senioridade:{senioridade} \n Data de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [145]:
################################################
# --- Agente 2: Planejador de carreira e recrutamento --- #
################################################
def agente_planejador_de_carreira(profissao, area, senioridade, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um especialista em carreira e recrutamento muito gentil e simpático, com vasta experiência em ajudar candidatos a se prepararem para processos seletivos.
        Gere dicas para se destacar, considerando os seguintes tópicos:
             1. Quais habilidades e experiências o candidato deve enfatizar?
             2. Forneça até 2 dicas concisas sobre como o usuário pode se organizar para se candidatar as vagas (ex: adaptar currículo, pesquisar a empresa).
             3. Ao final cite dicas gerais de como se destacar no mercado de trabalho de acordo com a profissão do usuário
        """,
        description="Agente planejador especialista em carreira",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Profissão: {profissao} \n Area:{area}  \n Senioridade:{senioridade} \n Buscados: {lancamentos_buscados}"

    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [154]:
def obter_input_obrigatorio(mensagem_inicial, mensagem_erro):
    # Exibe a mensagem inicial para o usuário, solicitando a entrada.
    valor = input(f"\n{mensagem_inicial} ")

    # Inicia um loop 'while' que continuará executando enquanto a condição for verdadeira.
    # A condição 'not valor.strip()' verifica se a variável 'valor' está vazia ou contém apenas espaços em branco.
    # 'valor.strip()' remove os espaços em branco do início e do final da string.
    # Se após a remoção dos espaços, a string estiver vazia, a condição será 'True' e o loop continuará.
    while not valor.strip():
        # Exibe a mensagem de erro informando ao usuário que a entrada é obrigatória.
        print(f"{mensagem_erro}")

        # Solicita novamente a entrada do usuário, exibindo a mensagem inicial novamente.
        valor = input(f"\n{mensagem_inicial}")

    # Quando o loop 'while' terminar (ou seja, o usuário digitou algo que não é apenas espaço),
    # a função retorna o valor digitado, com quaisquer espaços em branco iniciais e finais removidos
    # usando o método 'strip()'.
    return valor.strip()

In [152]:
def exibir_resultados(profissao, area, senioridade):
    print(f"")

    data_de_hoje = date.today().strftime("%d/%m/%Y")

    resposta_buscador = agente_buscador_vagas_de_emprego(profissao, area, senioridade, data_de_hoje)
    print("\n 📝 Essas foram as vagas encontradas! \n")
    display(to_markdown(resposta_buscador))
    print("--------------------------------------------------------------")

    print("\n🔄 Processando resultados de planejamento...")
    resposta_planejador = agente_planejador_de_carreira(profissao, area, senioridade, resposta_buscador)
    print("\n 🚀 Algumas dicas para decolar em seu processo seletivo! 🚀 \n")
    display(to_markdown(resposta_planejador))
    print("--------------------------------------------------------------")

    return

In [169]:
def main():
  exibir_questionario = "1";

  while exibir_questionario == "1" :
    clear_output(wait=True)

    print("\n Olá, tudo bem por aí? 😊 Que bom te ter aqui!")
    print("\n Sou seu assistente virtual de carreira, pronto para te dar uma mãozinha especial na busca daquela vaga de emprego e te preparar para o sucesso nas entrevistas! 🎉")
    print("\n--------------------------------------------------------------")

    profissao = obter_input_obrigatorio("❓Me conta qual a profissão que te interessa encontrar vagas hoje? ", "/n Hmm, para que eu te ajude a achar as melhores vagas, preciso que me conte qual profissão você busca, tá bem? 😉 ")
    area = obter_input_obrigatorio("❓Legal! qual a área específica dentro dessa profissão você deseja atuar? (Por exemplo: UX, Gestão, Frontend, Backend) " , "É importante me informar qual área específica dentro dessa profissão deseja atuar!😊 ")
    senioridade = obter_input_obrigatorio("❓E qual o seu nível de experiência? (Por exemplo: pleno, sênior, júnior, estágio...) ", "Porfavor, digite seu nível de experiência para refinar a busca! 😊 ")

    print("\n🔄 Processando resultados...")
    exibir_resultados(profissao, area, senioridade)

    print("Se deseja procurar outra vaga, digite 1: \n")
    exibir_questionario = input("")

    if(exibir_questionario != '1'):
      print("\n Até a próxima! 😊")
      break;


main()




 Olá, tudo bem por aí? 😊 Que bom te ter aqui!

 Sou seu assistente virtual de carreira, pronto para te dar uma mãozinha especial na busca daquela vaga de emprego e te preparar para o sucesso nas entrevistas! 🎉

--------------------------------------------------------------

❓Me conta qual a profissão que te interessa encontrar vagas hoje?  programador

❓Legal! qual a área específica dentro dessa profissão você deseja atuar? (Por exemplo: UX, Gestão, Frontend, Backend)  fullstack

❓E qual o seu nível de experiência? (Por exemplo: pleno, sênior, júnior, estágio...)  junior

🔄 Processando resultados...


 📝 Essas foram as vagas encontradas! 



> Com base nas buscas realizadas, aqui estão algumas vagas de Programador Fullstack Júnior que correspondem aos seus critérios:
> 
> 1.  **Desenvolvedor Full Stack Web Júnior**
> 
> *   **Empresa:** Netvagas
> *   **Localização:** Curitiba, Paraná, Brasil
> *   **Descrição:** Vaga para Desenvolvedor Full Stack Web Júnior.
> *   [Link para a vaga](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFstjAtJVzsNVKyqq-Qd_wKXFoELYDk0HE_mt2qUAbm7eH4wul2cdN987zW-RxHGuhcFo4lzf34l6vmLJGXFbVCpyXUt-cb535bbyn3mwWyC1jkNVwdXRnTFekQuc6g7aGQqRyIjo-uwtdhl_9C)
> 
> 2.  **Desenvolvedor Full-Stack - Júnior - PHP+React**
> 
> *   **Empresa:** FIESC - SENAI/SC
> *   **Localização:** Florianópolis - SC (Teletrabalho)
> *   **Descrição:** Atuar no desenvolvimento e manutenção de sistemas web voltados para a área educacional, utilizando PHP (Symfony) no backend e React no frontend. Requisitos: Ensino médio completo com graduação em andamento ou técnico em andamento nas áreas de tecnologia da informação, ou cursos de qualificação na área dos conhecimentos específicos ou graduação completa; 6 meses de experiência em: Desenvolvimento de Softwares; Análise e manutenção de dados em banco de dados; Bibliotecas de front-end; Ferramentas de versionamento; PHP com Framework Symfony; Testes automatizados com PHPUnit; React com componentização e estilização com Tailwind e/ou Styled Components.
> *   [Link para a vaga](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEmi9dX_XQRgAErig9OPen9-kt2poRJoOf8Zi2yK1pueoFS9EsMXJHgN1FIDQ0OuGUIpTWIi-lmyuOjoQxKLMjUzpnntLyZeoIw7iySQEsfv_igoL2Uwbgu7k1DCiC3r6b5HWOOgN-gPJ74lLvdAshevfYvKdgK7AhbRbpTMpviMwta3kF0tZfY16mAZ-zNJveBWym-o8lbeoU4oadqyDkyOSlo)
> 
> 3.  **Desenvolvedor Full Stack Junior**
> 
> *   **Empresa:** Grupo Roma Brasil
> *   **Localização:** Belo Horizonte - MG (Presencial)
> *   **Descrição:** Desenvolvedor Full Stack Junior. Requisitos: Ensino Superior completo ou cursando em, Ciências da Computação ou Sistemas de informação; Experiência em desenvolvimento de Software sob medida; Conhecimentos sólidos em OOP, MVC e Integrações Rest; Conhecimentos sólidos na plataforma Microsoft Azure; Domínio profundo de pelo menos duas a três linguagens de programação populares (Frameworks, PHP, MYSQL, SQL Server, Laravel, API, etc.).
> *   [Link para a vaga](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHiruJR4QOLOUgWQ69MYaXeljJ-qcpa62J1iYpyn8GL7o7kaHIJDAuIudI7_6cJ6FLCGONf8POWC62XZg3JMd3sRsP6UIjTCabaRp7Cusq1jzGOZaYPa8Z-651nMx72R2RXuQFSALA7JbhWHhrlWpeCSh1sH3qGjfuAbPJtUWbWp5btY99Ttw==)
> 
> 4.  **Desenvolvedor Fullstack Júnior**
> 
> *   **Empresa:** LWSA
> *   **Localização:** Remoto
> *   **Descrição:** Oportunidade para Desenvolvedor Fullstack Júnior em um ambiente colaborativo e inovador. Habilidades Requeridas: PHP Laravel Vue.js HTML CSS JavaScript MySQL Git Docker.
> *   [Link para a vaga](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFKx9kzctL7qEM7GEPP4jdfzmQd8q8yTJ5hWabwrC7S2WpaBFc0S0R-2i0Ov4tUkmyz8VzTkKfxMYkIbozp5lRcZarN59GRP4ZzdmcaLcKKz03ntL5lKo1Oov-zHjrRMA6eTL4SOC9hJJKUjJ4w7Qyj94iXLhd_9kxz9qSHkkoGs1DINaG-YLMBDWJ2sxVtEzwS3-M=)
> 
> 5.  **Programador Full Stack Junior**
> 
> *   **Empresa:** Teknisa
> *   **Localização:** Belo Horizonte - MG (Híbrido)
> *   **Descrição:** Participar de todas as etapas de um processo de desenvolvimento de software, desde a concepção das funcionalidades em conjunto com o time de produto até a implementação, manutenção e versionamento dos projetos; Ter a proatividade em compartilhar seus conhecimentos, contribuindo com desenvolvimento profissional da equipe; Deverá levar em consideração aspectos relacionados à escalabilidade, performance, qualidade de código e arquitetura de software; Sugerir e aplicar melhorias no processo de desenvolvimento; Buscar constantemente aperfeiçoamento tecnológico, mantendo-se atualizado quanto as mais novas tecnologias e metodologias de desenvolvimentos utilizadas no mercado; Ser comprometido com a equipe.
> *   [Link para a vaga](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHKDbTRQUz4gmKCRom4H81kX561bGAw-eF3xVJJdGHb6Ml6VYuAjaHjBNpqHXfaZ3xGxx8o6BekR7BxhQwRwQDoWXAkGjO7cF0B6BZCh4RPdhMtbqTplhcPPkN99ow3tCWiy7Ku93oN4_MeLkp_bHpnmBji0oACoxZWVRtcx5o50fpjLasUSbAJ0MoXCa9MSxULpxgSZtiP8DnvIgFIfPfM4ClQ6mAZCdkH)
> 


--------------------------------------------------------------

🔄 Processando resultados de planejamento...

 🚀 Algumas dicas para decolar em seu processo seletivo! 🚀 



> Olá! Com prazer, vou te ajudar a se destacar como programador fullstack júnior nessas oportunidades.
> 
> **1. Habilidades e Experiências a Enfatizar:**
> 
> *   **Conhecimento em Front-end e Back-end:** Demonstre suas habilidades em linguagens como HTML, CSS, JavaScript (React, Vue.js), e em back-end com PHP (Laravel, Symfony), Node.js ou Python. As vagas mencionam essas tecnologias como diferenciais.
> *   **Banco de Dados:** Enfatize sua experiência com MySQL, SQL Server, ou outros bancos de dados NoSQL, mostrando familiaridade com modelagem de dados e otimização de consultas.
> *   **Experiência com Frameworks:** Destaque seu conhecimento em frameworks como Laravel, Symfony, React, Vue.js, mostrando como você pode acelerar o desenvolvimento e manter a qualidade do código.
> *   **Versionamento e Metodologias Ágeis:** Mostre que você sabe usar Git para versionamento de código e que está familiarizado com metodologias ágeis, como Scrum ou Kanban, para trabalho em equipe.
> *   **Cloud e DevOps:** Se tiver experiência com plataformas como Microsoft Azure ou Docker, mencione-as, pois o conhecimento em cloud e DevOps é cada vez mais valorizado.
> 
> **2. Dicas de Organização para Candidaturas:**
> 
> *   **Currículo Adaptado:** Ajuste seu currículo para cada vaga, destacando as habilidades e experiências mais relevantes para a descrição da vaga.
> *   **Portfólio Atraente:** Crie um portfólio online com seus projetos, incluindo uma descrição detalhada do que você fez e as tecnologias utilizadas.
> 
> **3. Dicas Gerais para se Destacar no Mercado de Trabalho como Programador Fullstack:**
> 
> *   **Aprendizado Contínuo:** O mercado de tecnologia está sempre mudando, então invista em cursos online, bootcamps e certificações para se manter atualizado com as últimas tendências.
> *   **Networking:** Participe de eventos da área, meetups e grupos online para conhecer outros profissionais, trocar experiências e ficar por dentro de novas oportunidades.
> *   **Projetos Pessoais:** Desenvolva projetos pessoais para aplicar seus conhecimentos, experimentar novas tecnologias e construir um portfólio que demonstre suas habilidades.
> *   **Habilidades de Comunicação:** Desenvolva suas habilidades de comunicação para apresentar suas ideias de forma clara e eficaz, tanto para outros desenvolvedores quanto para clientes e stakeholders.
> 
> Espero que estas dicas te ajudem a se destacar nas candidaturas e a conquistar a vaga de programador fullstack júnior que você deseja! Boa sorte!
> 


--------------------------------------------------------------
Se deseja procurar outra vaga, digite 1: 

0

 Até a próxima! 😊
